In [56]:
import numpy as np
import re
from sympy import symbols, init_printing, gcd, expand
from sympy.polys.domains import GF,ZZ,QQ
from sympy import isprime
from collections import defaultdict


def fullchar(wght, G):
    """
    Compute the full character of the representation of a Lie group (by type).

    Parameters
    ----------
    wght : tuple of int
        The highest weight of the representation.
    G : str
        The type of the Lie group (e.g., 'A2', 'G2', 'E8').

    Returns
    -------
    sage.interfaces.lie.LiEElement
        The full character of the representation.

    Examples
    -------
    >>> fullchar([1,0],"A2")
    1X[-1, 1] +1X[ 0,-1] +1X[ 1, 0]
    """
    return lie.W_orbit(lie.dom_char(wght, G), G)


def coeffs(p):
    """
    Takes a polynomial and returns its coefficients as a list of integers

    The expectation is that polynomials passed here are of type sage.interfaces.lie.LiEElement, and of the form cX[e1,e2,...,en] + dX[f1,...,fn] + ...
    Thus, to obtain the coefficients (c,d,...), we use regular expressions (regex) to extract the digits before the "X".

    Parameters
    ----------
    p : sage.interfaces.lie.LiEElement
        The polynomial whose coefficients we wish to extract

    Returns
    -------
    list of int
        A list of coefficients of the polynomial

    Examples
    -------
    >>> coeffs(fullchar([1,0],"A2")) #Note that fullchar([1,0],"A2") is 1X[-1, 1] +1X[0,-1] +1X[1, 0]
    [1, 1, 1]
    """
    pattern = r'([+-]?\d+)X\[' #looking for patterns of the form "(digit)X["
    matches = re.findall(pattern, str(p))
    coefficients = [int(num) for num in matches]

    return coefficients


def exponents(p):
    """
    Takes a polynomial and returns its exponents as a list of list of integers

    The expectation is that polynomials passed here are of type sage.interfaces.lie.LiEElement, and of the form cX[e1,e2,...,en] + dX[f1,...,fn] + ...
    Thus, to obtain the exponents [[e1,e2,...,en],[f1,...,fn]], we use regular expressions (regex) to extract the digits between the [ & ] brackets.

    Parameters
    ----------
    p : sage.interfaces.lie.LiEElement
        The polynomial whose exponents we wish to extract

    Returns
    -------
    list of list of int
        A list of list of exponents of polynomial

    Examples
    -------
    >>> exponents(fullchar([1,0],"A2")) #Note that fullchar([1,0],"A2") is 1X[-1, 1] +1X[0,-1] +1X[1, 0]
    [[-1, 1], [0, -1], [1, 0]]
    """
    pattern = r'X\[(.*?)\]' #looking for patterns of the form "X[(content)]"
    matches = re.findall(pattern, str(p))
    exponents = [list(map(int, match.split(','))) for match in matches]
    
    return exponents


class LengthError(Exception):
    """
    Exception raised when the input does not have the correct length.
    """
    pass


def extract_info_from_fundamental_characters(G, fundamental_weights):
    """
    Returns the fundamental characters of a group G, as well as their coefficients and their exponents.

    Parameters
    ----------
    G : str
        The type of the Lie group (e.g., 'A2', 'G2', 'E8').
    fundamental_weights: list of list of ints
        list of fundamental weights (essentially a rank(G) x rank(G) identity matrix)

    Returns
    -------
    3-tuple of lists of objects
        The first returned list of objects are the fundamental characters.
        The second returned list of objects contain the coefficients of the fundamental characters.
        The third returned list of objects contain the exponents of the fundamental characters.

    Examples
    -------
    >>> extract_info_from_fundamental_characters("A2", [[1,0],[0,1]])
    ([1X[-1, 1] +1X[ 0,-1] +1X[ 1, 0], 1X[-1, 0] +1X[ 0, 1] +1X[ 1,-1]], [[1, 1, 1], [1, 1, 1]], [[[-1, 1], [0, -1], [1, 0]], [[-1, 0], [0, 1], [1, -1]]])
    """
    fundamental_characters = []
    coefficient_list = []
    exponents_list = []

    for index, weight in enumerate(fundamental_weights):
        p = fullchar(weight, G)
        fundamental_characters.append(p)
        coefficient_list.append(coeffs(p))
        exponents_list.append(exponents(p))

    return fundamental_characters, coefficient_list, exponents_list


def weighted_dynkin_and_characteristic_input(rankG):
    """
    Obtains user input for how SL2 acts on simple roots, as well as the (prime) characteristic.

    For each copy of SL2 in G, the neutral element of SL2 (usually denoted H) acts on the fundamental roots as 0,1,2
    (Note that there is no check to ensure all values are 0,1,2.)
    The user should know ahead of time how the desired SL2 acts on the roots. The user is prompted to input this data .
    Moreover, the calculations in the script are dependent on the (prime) characteristic, so the user is asked to input this too.

    Parameters
    ----------
    rankG : int
        The rank of the Lie group G.

    Returns
    -------
    list of int, int
        A list of ints describing how H acts on the simple roots. As there are rankG simple roots, the returned list will be of length rankG.
        Also returns the (prime) characteristic the user wishes to work over.

    Example
    -------
    >>> weighted_dynkin_and_characteristic_input(3)
    Enter the value of H when acting on each simple root (order them as in Humphrey's), each separated by a space: 0 1 2
    Enter the characteristic of prime field you want the K-theory calculated over: 2
    ([0, 1, 2], 2)

    In this example, the user inputs are `0 1 2` for how H acts on the 3 simple roots, and `2` for the characteristic.
    """
    while True:
        try:
            Rval = tuple(int(item) for item in input("Enter the value of H when acting on each simple root (order them as in Humphrey's), each separated by a space: ").split())
            if len(Rval) != rankG:
                raise LengthError
            break
        except LengthError:
            print(f"You have input the wrong number of integers. Please insert {rankG} integers, each separated by space.")
        except ValueError:
            print("One or more of the inputs were not of type int. Please try again.")

    while True:
        try:
            char = int(input("Enter the characteristic of prime field you want the K-theory calculated over: "))
            if not isprime(char) and char !=0:
                raise ValueError
            break
        except ValueError:
            print("Please insert a prime or 0.")

    return Rval,char


def action_on_roots_to_weights(Rval, G, rankG):
    """
    Converts action on roots to action on weights via the Cartan matrix.

    Let C be the Cartan matrix of `G`. Then C is the change of basis matrix from a simple root basis to a fundamental weights basis
    (i.e. it tells you how to write simples roots as linear combinations of the fundamental weights).
    So, to obtain how H acts on the fundamental weights we apply C^{-1} to how H acts on the simple roots `Rval`

    Parameters
    ----------
    Rval : list of int
        A list of ints describing how H in SL2 acts on the simple roots.
    G: str
        The type of the group (e.g. "A2","F4")
    rankG: int
        The rank of the group G

    Returns
    -------
    list of int
        A list of ints describing how H acts on the fundamental weights

    Example
    -------
    >>> action_on_roots_to_weights([1,0],"A2", 2)
    [0.6666666666666666, 0.3333333333333333]
    """
    #We require the Cartan matrix, or rather its inverse, since (H acting on weights)^T = C^{-1}(H acting on  roots)^T
    C_hold = lie.Cartan(G)
    C = [[int(C_hold[i][j]) for j in range(1, rankG+1)] for i in range(1, rankG+1)]
    Cinv = np.linalg.inv(C)

    Wval_hold = Cinv.dot(np.array(Rval)) #matrix `Cinv` acting on `Rval` i.e. (Cinv)*(Rval)
    Wval = list(map(float, Wval_hold))

    return Wval


def restrict_to_SL2(Wval, fund_characters, fund_chars_coefficients, fund_chars_exponents):
    """
    Converts each fundamental character of G to a character of SL2 using information on how H in SL2 acts on the fundamental weights

    Parameters
    ----------
    Wval : list of int
        A list of ints describing how H in SL2 acts on the fundamental weights.
    fund_characters: list of objects
        A list containing the rank(G) fundamental characters of G
    fund_chars_coefficients: list of list of ints
        A list of coefficients for each of the fundamental characters of G
    fund_chars_exponents: list of list of list of ints
        A list of list of exponents for each fundamental character of G

    Returns
    -------
    list of objects
        A list containing the coefficients and exponents of each fundamental character, after restricting to SL2 using Wval.
    """
    restricted_polys = []
    for k, p in enumerate(fund_characters):
        restriction_exponents = [int(round(sum(Wval[j] * fund_chars_exponents[k][i][j] for j in range(rankG)))) for i in range(int(lie.length(p)))]

        # Pair the restriction_exponents and fund_polys_coeff_list by index and sort by exponents in descending order.
        coefficients_by_exponents = defaultdict(int)
        for exponent, coefficient in zip(restriction_exponents, fund_chars_coefficients[k]):
            coefficients_by_exponents[exponent] += coefficient

        min_exponent = min(restriction_exponents)
        max_exponent = max(restriction_exponents)
        sorted_exponents = list(range(max_exponent, min_exponent - 1, -1))
        sorted_coefficients = [coefficients_by_exponents[exp] for exp in sorted_exponents]

        restricted_polys.append([sorted_coefficients, sorted_exponents])

    return restricted_polys


def restricted_chars_as_elements_of_R_SL2(res_char_data, rankG):
    """
    Converts the data of the restricted characters into elements of the representation ring of SL2 (i.e. elements in Z[x])

    `res_char_list` contains lists of coefficients and exponents for each restricted fundamental character of G. These lists of 
    coefficients and exponents are converted into elements of R(SL2) = R[x] by first decomposing the characters into a sum of irreducibles,
    then substituting in the chracter of each irreducible. 

    Parameters
    ----------
    res_char_data : list of objects
        A list of coefficients and exponents for each restrictred fundamental character of G
    rankG: int
        The rank of the group G

    Returns
    -------
    list of objects
        A list containing the restricted characters (as elements of R(SL2)).
    """
    #Note: symbols and init_printing are from the sympy package
    x = symbols('x')
    init_printing(use_unicode=False, wrap_line=False)

    sl2_irred_polys = [1, x]  # e.g. where P_0 = 1, P_1 = x and use [P_1][P_n] = [P_{n-1}]+[P_{n+1}] to calculate further
    max_irred = max(poly[1][0] for poly in res_char_data)  #maximum of the restricted exponents in res_poly_data a.k.a the maximum irreducible that we will need.

    for i in range(2, max_irred + 1):
        irred_sl2_character = sl2_irred_polys[1] * sl2_irred_polys[i - 1] - sl2_irred_polys[i - 2]
        sl2_irred_polys.append(expand(irred_sl2_character))

    # Initialize char_polys_wrt_P_i_coeffs with zeros. We will write each restricted character as a sum of the P_i (i.e. as a sum of irreducibles).
    restricted_characters_wrt_sl2_irreds_coefficients = [[0] * (max_irred + 1) for _ in range(rankG)]

    # Decompose the restricted characters into a sum of sl2 irreds
    for i in range(rankG):
        coeffs, exponents = res_char_data[i]
        for j in range(len(coeffs)):
            coeff = coeffs[j]
            if coeff != 0:
                restricted_characters_wrt_sl2_irreds_coefficients[i][exponents[j]] += coeff
                for k in range(exponents[j] + 1):
                    coeffs[j + 2 * k] -= coeff

    # Compute the restricted characters as elements of R(SL2) = Z[x]
    restricted_chars_wrt_x = [sum(restricted_characters_wrt_sl2_irreds_coefficients[i][j] * sl2_irrep_polys[j] for j in range(max_irred + 1)) for i in range(rankG)]

    return restricted_chars_wrt_x


def gcd_of_polys_minus_dimension(restricted_characters, fundamental_weights, char):
    """
    Find gcd of the restricted characters minus their dimensions over a given characteristic

    Parameters
    ----------
    restricted_characters : list of objects
        list of polynomials in 'x'
    fundamental_weights: list of list of integers
        list of fundamental weights, used to calculate dimension of fundamental representations
    char: int
        The characterstic of the prime field we are working over

    Returns
    -------
    object
        The gcd of the passed polynomials minus their dimensions.
    """
    Y = restricted_characters[0] - int(lie.dim(fundamental_weights[0], G))
    if rankG > 1:
        for i in range(1, rankG):
            if char == 0:
                Y = gcd(restricted_characters[i] - int(lie.dim(fundamental_weights[i], G)), Y, domain=QQ)
            elif char > 0:
                Y = gcd(restricted_characters[i] - int(lie.dim(fundamental_weights[i], G)), Y, domain=GF(char))
    return Y


def print_result_and_go_again(G, Rval, Y, char):
    """
    Prints the result and stores user input on whether to go again or not.

    Parameters
    ----------
    G : str
        The type and rank of group (e.g. "A2")
    Rval: list of ints
        The value of the action of H in SL2 on the simple roots of G
    Y: sympy.core.add.Add (a polynomial)
        The polynomial to quotient Z[X] by.
    char: int
        The characterstic of the prime field we are working over

    Returns
    -------
    str
        A string indicating if the script runs again. 'n' for NO, anything else for YES

    Examples
    --------
    >>> print_result_and_go_again("A2", [2,2], x**2 - 1, 3)
    ------------------------------------------------------------

    Let F denote the prime field of characteristic 3

    K^{0,0}(A2/SL2) ⊗ F, when H acts as (2, 2) on the simple roots is:

    F[x]/(x**2 - 1)

    ------------------------------------------------------------

    Would you like to input another weighted Dynkin diagram? (type 'n' or 'N' for NO, type anything else for YES)
    """
    print(f"\n{'-'*60}\n")
    print(f"Let F denote the prime field of characteristic {char}\n")
    print(f"K^{{0,0}}({G}/SL2) \u2297 F, when H acts as {Rval} on the simple roots is:\n")
    print(f"F[x]/({Y})\n")
    print(f"{'-'*60}\n")

    print("Would you like to input another weighted Dynkin diagram? (type 'n' or 'N' for NO, type anything else for YES) ")
    go_again = input()
    print(f"{'-'*60}\n")
    return go_again.lower()


if __name__ == "__main__":
    G = input("Enter group name as 'Letter + number', e.g. A2, A4, F4, G2, E6, ... ")
    G = G[0].upper()+G[1:]
    rankG = int(lie.Lie_rank(G))

    # Produce a list of lists of fundamental weights, there will be rankG of these (this is eseentially a rank(G) x rank(G) identity matrix)
    fundamental_weights = [[1 if i == j else 0 for j in range(rankG)] for i in range(rankG)]

    #Obtain fundamental characters, their coefficients, and their exponents for the given group G.
    fund_characters, fund_characters_coefficients, fund_characters_exponents = extract_info_from_fundamental_characters(G, fundamental_weights)

    go_again = 'y'
    while go_again != 'n':
        Rval, char = weighted_dynkin_and_characteristic_input(rankG)

        # Rval tells us how H in SL2 acts on the simple roots. However, we need to know how H acts on the fundamental weights.
        Wval = action_on_roots_to_weights(Rval, G, rankG)

        # Restrict each fundamental character to SL2, using Wval, which tells us how H acts on the fundamental weights.
        restricted_chracters_coeffs_and_exponents = restrict_to_SL2(Wval, fund_characters, fund_characters_coefficients, fund_characters_exponents)

        # Write each list of coefficients and exponents of the restricted fundamental characters (above) as an element of R[SL2] = Z[x]
        restricted_characters_wrt_x = restricted_chars_as_elements_of_R_SL2(restricted_chracters_coeffs_and_exponents, rankG)

        # Compute the gcd of the restricted fundamental characters (expressed as elements of R[SL2] = Z[x]) - their dimensions, over the given characteristic
        Y = gcd_of_polys_minus_dimension(restricted_characters_wrt_x, fundamental_weights, char)

        #prints result and asks to go_again? 'n' for NO, anything else for YES
        go_again = print_result_and_go_again(G, Rval, Y, char)

Enter group name as 'Letter + number', e.g. A2, A4, F4, G2, E6, ...  a3

Enter the value of H when acting on each simple root (order them as in Humphrey's), each separated by a space:  2 2 2

Enter the characteristic of prime field you want the K-theory calculated over:  0


------------------------------------------------------------

Let F denote the prime field of characteristic 0

K^{0,0}(A3/SL2) ⊗ F, when H acts as (2, 2, 2) on the simple roots is:

F[x]/(x - 2)

------------------------------------------------------------

Would you like to input another weighted Dynkin diagram? (type 'n' or 'N' for NO, type anything else for YES) 


 n

------------------------------------------------------------

